In [1]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 24px;
}

div.output_area pre {
    font-size: 24px;
}
</style>

In [2]:
print("hello")

hello


In [3]:
import numpy as np
import math
import tensorflow as tf
import h5py
import sys
import os

sys.path.append('../../')
import deepbayesHF

import tensorflow as tf
from deepbayesHF import optimizers
from deepbayesHF import PosteriorModel
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

######## OPTIONS #########
ver = 6              # Neural network version
table_ver = 6        # Table Version
hu = 25              # Number of hidden units in each hidden layer in network
saveEvery = 1000     # Epoch frequency of saving
totalEpochs = 3000   # Total number of training epochs
trainingDataFiles = "./TrainingData/HCAS_rect_TrainingData_v%d_pra%d_tau%02d.h5" # File format for training data
nnetFiles  = "./networks/HCAS_rect_v%d_pra%d_tau%02d_%dHU.nnet" # File format for .nnet files
modelFiles = "./models/HCAS_rect_v%d_pra%d_tau%02d_%dHU.ckpt" # File format for .nnet files
tbFiles = "./tensorboard/HCAS_rect_v%d_pra%d_tau%02d_%dHU"
##########################

# Custom tensorflow session. Sets up training with either a cpu, gpu, or multiple gpus
def get_session(gpu_ind,gpu_mem_frac=0.45):
    """Create a session that dynamically allocates memory."""
    if gpu_ind[0]>-1:
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(np.char.mod('%d', gpu_ind))
        config = tf.ConfigProto(device_count = {'GPU': len(gpu_ind)})
        config.gpu_options.per_process_gpu_memory_fraction = gpu_mem_frac
        session = tf.Session(config=config)
    else:
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
        session = tf.Session()
    return session

# Function to compute network accuracy given. However, this does not 
# add the online costs that were used to generate the correct minimum cost index, so
# these accuracies are only an estimate
# Note from MRW: This doesnt make sense as a metric...
def custAcc(y_true,y_pred):
    maxesPred = tf.argmax(y_pred,axis=1)
    inds = tf.argmax(y_true,axis=1)
    diff = tf.cast(tf.abs(inds-maxesPred),dtype='float64')
    ones = tf.ones_like(diff,dtype='float64')
    zeros= tf.zeros_like(diff,dtype='float64')
    l = tf.where(diff<0.5,ones,zeros)
    return tf.reduce_mean(l)


# The previous RA should be given as a command line input
if len(sys.argv) > 2:
    pra = 0 #int(sys.argv[1])
    tau = 20 #int(sys.argv[2])
    gpu = -1
    print("Posteriors/HCAS_BNN_%s_%s"%(pra, tau))
    if len(sys.argv)>3:
        gpu = int(sys.argv[3])

    print("Loading Data for HCAS, pra %02d, Network Version %d" % (pra, ver))
    f       = h5py.File(trainingDataFiles % (table_ver,pra,tau),'r')
    X_train = np.array(f['X']).astype('float32')
    y_train       = np.array(f['y']).astype('float32')
    Q       = np.array(f['y']).astype('float32')
    means = np.array(f['means'])
    ranges=np.array(f['ranges'])
    mins = np.array(f['min_inputs'])
    maxes = np.array(f['max_inputs'])

    min_inputs      = ",".join(np.char.mod('%f', mins))
    max_inputs     = ",".join(np.char.mod('%f', maxes))
    means      = ",".join(np.char.mod('%f', means))
    ranges     = ",".join(np.char.mod('%f', ranges))
    
    N,numInputs = X_train.shape
    N,numOut = Q.shape
    X_train = tf.cast(X_train, dtype=tf.float64)
    y_train = tf.argmax(y_train,axis=1)

    print("Setting up Model")

    #loss = tf.keras.losses.SparseCategoricalCrossentropy()
    y_train = tf.one_hot(y_train, depth=numOut)
    y_train = tf.cast(y_train, dtype=tf.float64)
    loss = tf.keras.losses.CategoricalCrossentropy()
    # Use Keras to define our model
    WIDTH = 125
    model = Sequential()
    model.add(Dense(WIDTH, activation="relu", input_shape=(1, numInputs)))
    model.add(Dense(numOut, activation="softmax"))
    print("Ins and Outs: ", numInputs, numOut)
    # Use deepbayesHF to define our optimizer
    optimizer = optimizers.PriorHamiltonianMonteCarlo()
    bayes_model = optimizer.compile(model, loss_fn=loss, learning_rate = 0.1, mh_burn=False,
                          epochs=25, decay=0.3, steps = 30, b_steps = 25, burn_in=20,
                          rob_lam=0.8, classes = 5, path = "Posteriors/HMC_HCAS_BNN_%s_%s"%(pra, tau),
                          inflate_prior=500., mode='classification') # select optimizer and set learning rate

    # Train and save BNN using deepbayesHF
    #bayes_model.train(np.asarray([X_train]), y_train, np.asarray([X_train]), np.asarray([y_train]))
    bayes_model.constraint_train(np.asarray([X_train]), np.asarray([y_train]), 
                            np.asarray([X_train])-0.012, np.asarray([X_train])+0.012,
                            np.asarray([X_train]), np.asarray([y_train]))
    bayes_model.finalize_chain()
    y_pred = bayes_model.predict(X_train)

    #bayes_model.save("Posteriors/ROB_HCAS_BNN_%s_%s"%(pra, tau))


Posteriors/HCAS_BNN_0_20
Loading Data for HCAS, pra 00, Network Version 6
Setting up Model
Ins and Outs:  3 5
This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
deepbayes: Using implicit prior
(3, 125) 12.909944487358056
(125, 5) 2.0
deepbayes: Using implicit prior
(3, 125) 12.909944487358056
(125, 5) 2.0
(1, 53792, 3) (1, 53792, 5)
Still in Burn-In state of Markov chain.


/home/matker/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
Numerical Integration: 100%|██████████| 25/25 [00:10<00:00,  2.48it/s]


Current  U:  tf.Tensor(11.800755, shape=(), dtype=float32)
Proposed U:  tf.Tensor(11.41677, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(1.1683564, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 1, loss: 11.601, acc: 0.511, val_loss: 1.071, val_acc: 0.665
Still in Burn-In state of Markov chain.


Numerical Integration: 100%|██████████| 25/25 [00:10<00:00,  2.48it/s]


Current  U:  tf.Tensor(11.41677, shape=(), dtype=float32)
Proposed U:  tf.Tensor(11.385851, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(1.0059974, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 2, loss: 11.536, acc: 0.643, val_loss: 1.014, val_acc: 0.657
Still in Burn-In state of Markov chain.


...

Current  U:  tf.Tensor(13.626598, shape=(), dtype=float32)
Proposed U:  tf.Tensor(13.614344, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(1.011824, shape=(), dtype=float32)
ACCEPTED
saving with first value:  tf.Tensor(-1.1094762, shape=(), dtype=float32)
loaded with first value:  tf.Tensor(-1.1094762, shape=(), dtype=float32)
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 20, loss: 12.121, acc: 0.664, val_loss: 2.460, val_acc: 0.736
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 30/30 [00:11<00:00,  2.52it/s]


Current  U:  tf.Tensor(13.614344, shape=(), dtype=float32)
Proposed U:  tf.Tensor(13.711512, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.89977676, shape=(), dtype=float32)
ACCEPTED
saving with first value:  tf.Tensor(-1.1111356, shape=(), dtype=float32)
loaded with first value:  tf.Tensor(-1.1111356, shape=(), dtype=float32)
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 20, loss: 12.120, acc: 0.664, val_loss: 2.557, val_acc: 0.721
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 30/30 [00:11<00:00,  2.51it/s]


Current  U:  tf.Tensor(13.711512, shape=(), dtype=float32)
Proposed U:  tf.Tensor(13.682532, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(1.0280489, shape=(), dtype=float32)
ACCEPTED
saving with first value:  tf.Tensor(-1.1122907, shape=(), dtype=float32)
loaded with first value:  tf.Tensor(-1.1122907, shape=(), dtype=float32)
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 20, loss: 12.122, acc: 0.664, val_loss: 2.529, val_acc: 0.725
Out of Burn-In state. Generating samples from the chain
Must call finlize_chain() after this to save/use posterior samples
Finalized chain 27 26


In [4]:
bayes_model.save("Posteriors/HMC_HCAS_BNN_%s_%s"%(pra, tau))

('classes', 5)
('batch_size', 64)
('learning_rate', 0.011764705882352941)
('decay', 0.3)
('epochs', 26)
('inflate_prior', 500.0)
('input_noise', 0.0)
('robust_train', 0)
('epsilon', 0.1)
('robust_lambda', 0.8)
('loss_monte_carlo', 2)
('input_upper', inf)
('input_lower', -inf)
('burn_in', 20)
('m_burn', 0.1)
('b_steps', 25)
('steps', 30)
('preload', -1)
('m', 0.1)
('iterate', 26)
('global_iterate', 26)
('max_eps', 0.1)
('_learning_rate', 0.1)


In [5]:
bmodel = PosteriorModel("Posteriors/HMC_HCAS_BNN_%s_%s"%(pra, tau))

[INFO] BayesKeras: Attempting to load a sample based posterior
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 125)            500       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 5)              630       
Total params: 1,130
Trainable params: 1,130
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


In [14]:
y_pred_1 = bmodel.predict(X_train)

y_pred_2 = bayes_model.predict(X_train)

sampling 3
sampling 5
sampling 10
sampling 3
sampling 4
sampling 20
sampling 8
sampling 23
sampling 20
sampling 23
sampling 5
sampling 10
sampling 6
sampling 22
sampling 24
sampling 23
sampling 8
sampling 19
sampling 11
sampling 8
sampling 10
sampling 3
sampling 7
sampling 1
sampling 19
sampling 10
sampling 6
sampling 13
sampling 2
sampling 14
sampling 3
sampling 12
sampling 9
sampling 15
sampling 15


In [15]:
y_pred_max = np.max(y_pred_1, axis=1)
y_pred_1 = np.argmax(y_pred_1, axis=1)
y_train = np.argmax(y_train, axis=1)

In [8]:
#print(np.asarray(bmodel.sample()[0][0][0]))
#print(np.asarray(bayes_model.posterior_samples[6][0][0][0]))

In [17]:
corr = 0
for i in range(len(y_pred_1)):
    #print(y_pred[i], y_train[i])
    if(y_pred_1[i] == y_train[i]):
        print(y_pred_max[i])
        corr += 1
print(corr)
print(corr/len(y_pred_1))

0.9999989
0.9999989
0.9999989
0.9999989
0.9999989
0.9999989
0.9999988
0.9999987
0.99999845
0.99999803
0.99999803
0.99999774
0.9999971
0.68106824
0.9523031
0.9993239
0.9999933
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999987
0.9999987
0.99999857
0.99999857
0.99999857
0.99999857
0.99999857
0.9999983
0.99999803
0.9999979
0.9999979
0.99999774
0.9999971
0.68257254
0.95518625
0.99937546
0.9999937
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999845
0.99999845
0.99999845
0.9999983
0.9999983
0.9999983
0.9999983
0.9999979
0.9999978
0.9999978
0.9999978
0.9999976
0.99999696
0.6835451
0.9579458
0.9994119
0.9999939
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999979
0.9999979
0.9999978
0.9999978
0.9999978
0.9999978
0.9999978
0.9999978
0.99999774
0.99999774
0.99999774
0.9999975
0.99999684
0.6833316
0.9598152
0.99944043
0.9999942
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999774
0.99999774
0.99999774
0.99999774
0.99999774
0.99999774
0.99999774
0.9999976
0.9999976
0.9999975
0.9999975
0.99999726
0.9999967
0.6810565
0.96

0.9998887
0.9998891
0.9998895
0.9998899
0.9998903
0.9998909
0.99989134
0.99989146
0.9998924
0.9998935
0.9998944
0.9998919
0.99985397
1.0
1.0
0.999882
0.99988204
0.9998823
0.9998823
0.9998824
0.99988264
0.9998832
0.9998837
0.999884
0.99988467
0.99988467
0.99988586
0.99988663
0.99988645
0.99988186
0.9998281
1.0
1.0
1.0
0.99987453
0.9998749
0.9998749
0.999875
0.9998752
0.99987566
0.99987596
0.9998764
0.99987674
0.9998773
0.9998775
0.99987805
0.9998788
0.9998777
0.99987006
1.0
1.0
1.0
1.0
0.9998666
0.9998667
0.9998669
0.999867
0.99986714
0.9998674
0.99986744
0.9998681
0.99986845
0.9998687
0.99986875
0.9998695
0.99986964
0.9998667
0.99985546
0.88002807
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99985766
0.9998579
0.9998578
0.9998579
0.99985796
0.99985856
0.99985874
0.99985886
0.9998594
0.9998598
0.9998598
0.99986005
0.99985963
0.99985415
0.99983716
0.9072971
0.9988426
0.99998814
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9998483
0.9998486
0.9998487
0.9998486
0.9998487
0.99984896
0.999849
0.9998497
0.9998

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99935955
0.9993594
0.99935925
0.99935955
0.9993594
0.999359
0.9993587
0.9993575
0.99935585
0.9993536
0.99935323
0.9993439
0.999326
0.999246
0.9990429
0.6820984
0.99334544
0.9999261
0.9999991
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.999299
0.99929875
0.9992985
0.99929863
0.9992981
0.99929714
0.99929637
0.999294
0.99929047
0.9992859
0.99928546
0.9992689
0.9992398
0.9991171
0.7593954
0.9955802
0.9999515
0.9999996
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.999233
0.9992323
0.9992315
0.99923074
0.9992297
0.9992279
0.99922574
0.9992209
0.9992146
0.99920684
0.9992061
0.99917895
0.9991338
0.99895084
0.8248877
0.99712783
0.9999684
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9991567
0.99915534
0.99915403
0.9991525
0.9991509
0.9991477
0.9991438
0.9991354
0.9991255
0.999113
0.9991116
0.99906987
0.99900305
0.9987345
0.8745324
0.9981441
0.99998
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9990758
0.99907357
0.9990714
0.999069
0.9990665
0.99906135

0.9996205
0.9996209
0.99962014
0.99961776
0.9996133
0.9996125
0.99959075
0.9995453
0.99933404
0.80316997
0.98942554
0.999869
0.9999984
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9995467
0.99954814
0.9995492
0.9995503
0.9995516
0.9995534
0.99955523
0.9995576
0.9995584
0.9995575
0.9995572
0.9995453
0.9995137
0.99934673
0.7950818
0.98873997
0.9998667
0.99999845
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99946356
0.9994654
0.9994675
0.99946946
0.9994713
0.9994745
0.9994776
0.99948317
0.9994871
0.9994895
0.9994897
0.99948686
0.999467
0.9993383
0.78624964
0.98848253
0.9998657
0.99999857
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9993649
0.99936765
0.99937046
0.9993732
0.99937546
0.99938047
0.9993852
0.9993939
0.9994011
0.9994071
0.99940765
0.99941385
0.99940544
0.99931115
0.77781636
0.9884545
0.9998665
0.99999857
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9992496
0.9992531
0.9992567
0.9992602
0.99926347
0.9992702
0.9992764
0.9992886
0.9992994
0.99930865
0.99930954
0.9993249
0.9993273
0.99926347


0.928559
0.9286415
0.9287187
0.9287884
0.9289072
0.9289978
0.92907137
0.9289873
0.92871594
0.9286774
0.9270367
0.923466
0.90539646
0.8635651
0.63069487
0.9596442
0.9994395
0.99999326
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.92414445
0.9241407
0.92412764
0.9241055
0.92407346
0.9239795
0.92385036
0.92348576
0.92290854
0.9220861
0.9219893
0.9187074
0.9127738
0.8869112
0.83084106
0.55534863
0.8174057
0.9725053
0.9996555
0.99999607
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.918472
0.918345
0.9182059
0.9180543
0.9178893
0.91752213
0.917099
0.9160626
0.91474295
0.9131064
0.9129243
0.90752494
0.89840126
0.8627882
0.79033715
0.8677562
0.98211026
0.9997845
0.9999975
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9119418
0.91170985
0.9114618
0.91119576
0.91091233
0.9102907
0.9095927
0.90794885
0.9059313
0.9034886
0.903218
0.89505875
0.88167524
0.8332205
0.7403343
0.91135514
0.98881555
0.99986243
0.9999984
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9041749
0.9037703
0.903

0.5442575
0.53879106
0.5134804
0.4591797
0.99987096
0.9999987
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.54005015
0.5401705
0.5402785
0.5403833
0.54047745
0.54062426
0.5407123
0.54071724
0.5403922
0.53958213
0.5394755
0.5353645
0.5272702
0.49516323
0.43376687
0.9999058
0.9999992
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.53202707
0.53203297
0.5320228
0.53199625
0.5319556
0.531789
0.5315388
0.53078294
0.5295519
0.5277706
0.5275651
0.52089643
0.5099062
0.4690478
0.39727646
0.9999336
0.99999946
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.5195082
0.51919585
0.51886284
0.5185084
0.5181318
0.51730967
0.51638734
0.5141999
0.51154476
0.5083713
0.5080226
0.4975042
0.48149577
0.43156216
0.99995315
0.9999996
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.49912024
0.49857128
0.49800077
0.49740365
0.49677914
0.49544486
0.4939917
0.49070108
0.4866987
0.4819773
0.48146105
0.46694162
0.4469339
0.38773265
0.9963211
0.9999663
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


1.0
0.8879533
0.88907593
0.8836934
0.85530853
0.99813145
0.999977
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.89552134
0.89448076
0.88519317
0.49922845
0.8817339
0.99853516
0.99998254
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.89851665
0.89584255
0.8829901
0.769011
0.553753
0.9044533
0.9988711
0.9999876
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8940594
0.87577915
0.73774874
0.60904986
0.9240956
0.9992072
0.9999915
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.89066863
0.865377
0.69739753
0.67097205
0.94376135
0.99945295
0.9999939
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8839693
0.848169
0.63647354
0.7513945
0.96332455
0.99962115
0.99999577
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.8707908
0.81914824
0.5495483
0.83129835
0.9773099
0.99975085
0.9999974
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.4750897
0.47090003
0.470435
0.45683232
0.43619996
0.36491013
0.9768319
0.9973371
0.99996185
0.99999946
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.

1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9978921
0.99788994
0.9978873
0.9978846
0.9978814
0.9978747
0.99786663
0.9978463
0.9978194
0.99778444
0.99778026
0.99764675
0.9974001
0.99622256
0.59261787
0.9892136
0.999846
0.9999979
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.98597294
0.9859534
0.9859328
0.9859119
0.98588955
0.98584276
0.9857917
0.98567593
0.9855361
0.9853591
0.9853397
0.9847412
0.98373324
0.9790122
0.63211024
0.93380314
0.9992652
0.99999213
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.98869413
0.98867786
0.9886611
0.98864365
0.9886259
0.9885883
0.988548
0.9884566
0.98834926
0.98822325
0.9882094
0.987791
0.9871001
0.9839399
0.52868915
0.9031271
0.9988803
0.99998826
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99082655
0.9908146
0.99080247
0.9907898
0.99077636
0.9907491
0.9907204
0.9906564
0.9905824
0.9904954
0.99048615
0.9902015
0.9897341
0.98764277
0.8598279
0.9982789
0.9999819
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99253684
0.9925283
0.9925198
0.9925

0.999988
0.999988
0.99998814
0.99998814
0.99998814
0.99998814
0.99998814
0.99998826
0.9999883
0.9999883
0.99998826
0.999988
0.9615939
0.99924666
0.99998695
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99998766
0.99998766
0.99998766
0.99998766
0.99998766
0.99998766
0.99998766
0.9999878
0.99998766
0.99998766
0.9999878
0.99998766
0.9999879
0.9999878
0.9999875
0.96486837
0.9993042
0.99998736
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998724
0.99998736
0.99998736
0.99998724
0.96813375
0.99936515
0.9999888
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999866
0.9999866
0.9999866
0.9999868
0.9999868
0.9999867
0.9999867
0.9999867
0.9999867
0.9999867
0.9999867
0.9999867
0.9999866
0.9999867
0.5117759
0.97127324
0.99943835
0.9999903
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99998593
0.99998593
0.99998593
0.99998593
0.99998593
0.99998593
0.99998593
0.99998593
0.99998605
0.99998605
0.999

0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.6545537
0.9835615
0.99968445
0.9999951
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.64532447
0.98385775
0.9996873
0.99999434
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999997
0.9999997
0.9999997
0.9999997
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.9999996
0.56129193
0.97729635
0.9995541
0.9999921
0.9999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9686845
0.99936205
0.9999888
0.99999976
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.99999976
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9999997
0.9568259
0.999085
0.99998367
0.99999976
1.0
1.0
1

In [18]:
corr = 0
y_pred_max = 
for i in range(len(y_pred_2)):
    #print(y_pred[i], y_train[i])
    if(y_pred_2[i] == y_train[i]):
        corr += 1
print(corr)
print(corr/len(y_pred_2))

SyntaxError: invalid syntax (<ipython-input-18-7d71b5d6e21f>, line 2)